_Note: Prior to storing a jupyter notebook in git, use_
```python3 -m pip install nb-clean```
and in the git directory execute:
```nb-clean add-filter```

# Tanzu Build Service Install

```Note: locally using .envrc. You need to ensure the variables in the following commands are exported```

```
export TKGI_API=
export TKGI_USERNAME= 
export TKGI_PASSWORD=
export TKGI_PATH_TO_CA_CERT=
export TBS_TARGET_CLUSTER=
export PIVOTAL_DOCKER_LOGIN_USERNAME=
export PIVOTAL_DOCKER_LOGIN_PASSWORD=
export HARBOR_DOMAIN=
export HARBOR_PROJECT=
export HARBOR_USER_NAME=
export HARBOR_PASSWORD=
export HARBOR_CA_CERT='-----BEGIN CERTIFICATE-----
-----END CERTIFICATE-----'
```

In [ ]:
eval "$(direnv hook bash)"

## Ensure tooling is installed

In [ ]:
kapp --version

In [ ]:
ytt --version

In [ ]:
kbld --version

## Get bits from Tanzu Network

Pull down the latest TBS ```https://network.pivotal.io/products/build-service/#/releases/768025```

Pull down the latest kp cli from TBS page

In [ ]:
chmod +x ~/Downloads/kp-darwin-0.1.3

In [ ]:
mv ~/Downloads/kp-darwin-0.1.3   /usr/local/bin/kp

In [ ]:
kp version

Pull down the latest build-service-< x >.tar

In [ ]:
mkdir build-service-bits

In [ ]:
tar xvf build-service-1.0.3.tar -C /tmp

Ensure docker cli is installed

In [ ]:
docker version

Pull down the latest build service dependencies from https://network.pivotal.io/products/tbs-dependencies/#/releases/808249

## Move Images to local repo

In [ ]:
docker login $HARBOR_DOMAIN -u $HARBOR_USER_NAME -p $HARBOR_PASSWORD

In [ ]:
docker login registry.pivotal.io -u $PIVOTAL_DOCKER_LOGIN_USERNAME -p $PIVOTAL_DOCKER_LOGIN_PASSWORD

In [ ]:
kbld relocate -f /tmp/images.lock --lock-output /tmp/images-relocated.lock --repository $HARBOR_DOMAIN/$HARBOR_PROJECT/core

## Install Build Service

### Login to target cluster

In [ ]:
pks login -a $TKGI_API -u $TKGI_USERNAME -p $TKGI_PASSWORD --ca-cert $TKGI_PATH_TO_CA_CERT

In [ ]:
echo $TKGI_PASSWORD | pks get-credentials $TBS_TARGET_CLUSTER

In [ ]:
kubectl config use-context $TBS_TARGET_CLUSTER

### Create Storage Class

In [ ]:
kubectl apply -f default-vsphere-storage-class.yml

In [ ]:
kubectl patch storageclass vsphere-fast -p '{"metadata": {"annotations":{"storageclass.kubernetes.io/is-default-class":"true"}}}' 

### Install Build Service

In [ ]:
ytt -f /tmp/values.yaml \
    -f /tmp/manifests/ \
    -v ca_cert_data="$HARBOR_CA_CERT" \
    -v docker_repository="$HARBOR_DOMAIN/$HARBOR_PROJECT" \
    -v docker_username="$HARBOR_USER_NAME" \
    -v docker_password="$HARBOR_PASSWORD" \
    | kbld -f /tmp/images-relocated.lock -f- \
    | kapp deploy -a tanzu-build-service -f- -y

### Import buildpacks

In [ ]:
kp import -f./descriptor-100.0.56.yaml --registry-verify-certs false

# Verify Installation

In [ ]:
kp clusterbuilder list

In [ ]:
kp clusterstack list

In [ ]:
kp clusterstack status base

In [ ]:
kp clusterstore list

In [ ]:
kp clusterstore status default

## Verify Build Process

### Setup secret for registry to which we will be publishing

Run this command at the command prompt and enter Harbor password when prompted: 
```kp secret create test-app-registry-creds --registry $HARBOR_DOMAIN --registry-user $HARBOR_USER_NAME```

In [ ]:
kp secret list

### Build Sample

Create a sample project in Harbor to build the image. Examples below use the project name: ```test-java-app-from-builder```

In [ ]:
kp image create sample-image --tag $HARBOR_DOMAIN/test-java-app-from-builder/sample --git https://github.com/buildpacks/samples --sub-path ./apps/java-maven --wait

In [ ]:
docker pull $HARBOR_DOMAIN/test-java-app-from-builder/sample:latest

In [ ]:
docker run -itd -p 8080:8080 --name tbs-sample-app harbor.management.pks.erds.io/test-java-app-from-builder/sample:latest

Go to http://localhost:8080 to see the app

In [ ]:
docker kill "$(docker ps -aqf 'name=^tbs-sample-app$')"

In [ ]:
docker rm "$(docker ps -aqf 'name=^tbs-sample-app$')"

### Deploy to K8s

In [ ]:
kubectl apply -f sample-deployment.yaml

In [ ]:
kubectl apply -f sample-service.yaml

In [ ]:
kubectl get svc sample-app

### Clean up

In [ ]:
kubectl delete svc sample-app

In [ ]:
kubectl delete deployment sample-app